In [ ]:
import numpy as np 
import random
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import cv2
from shutil import copyfile
from keras.callbacks import LearningRateScheduler
import keras
from keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D, \
    GlobalMaxPooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, \
    AveragePooling2D, Reshape, Permute, multiply, ZeroPadding2D, Dropout, LeakyReLU
from keras import applications
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
import warnings
from keras.models import Model, Sequential, load_model, model_from_json
from keras import layers
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from keras.layers import Input, Conv2D, multiply, LocallyConnected2D, Lambda, AvgPool2D, Reshape
from keras.applications import VGG16
from keras.applications import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.utils.generic_utils import get_custom_objects

# Swish Activation Function
def swish(x):
    return K.sigmoid(x) * x

get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate

# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [ ]:
img_rows, img_cols = (334,334)
train_batchsize = 16
val_batchsize = 16

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/train_images',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bicubic')
 
validation_generator = validation_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')

img_rows, img_cols = (224, 224)
train_batchsize = 16
val_batchsize = 16

train_datagen_crop = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen_crop = ImageDataGenerator(rescale=1./255)

train_generator_crop = train_datagen_crop.flow_from_directory(
        '../input/bird-dasat/bird_dataset/train_images_cropped',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bicubic')
 
validation_generator_crop = validation_datagen_crop.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images_cropped',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')

In [ ]:
shapes_resnet = [(9, 9, 1536), (5, 5, 1536)]
shapes_inception = [(9, 9, 2048), (5, 5, 2048)]

In [ ]:
def model_attention(shape, size=(334,334,3), num_unfreeze=0):
    base_pretrained_model = InceptionV3(input_shape =size, 
                              include_top = False, weights = 'imagenet')
    for layer in base_pretrained_model.layers[:len(base_pretrained_model.layers)-num_unfreeze]:
        layer.trainable = False
    for layer in base_pretrained_model.layers[len(base_pretrained_model.layers)-num_unfreeze:]:
        layer.trainable = True
    model = Sequential()
    pt_features = Input(shape , name = 'feature_input')
    pt_depth = shape[-1]
    bn_features = BatchNormalization()(pt_features)
    # here we do an attention mechanism to turn pixels in the GAP on an off
    attn_layer = Conv2D(128, kernel_size = (1,1), padding = 'same', activation = 'elu')(bn_features)
    attn_layer = Conv2D(32, kernel_size = (1,1), padding = 'same', activation = 'elu')(attn_layer)
    attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'elu')(attn_layer)
    attn_layer = AvgPool2D((2,2), strides = (1,1), padding = 'same')(attn_layer) # smooth results
    attn_layer = Conv2D(1, 
                        kernel_size = (1,1), 
                        padding = 'valid', 
                        activation = 'sigmoid')(attn_layer)
    # branch it to all channel
    up_c2_w = np.ones((1, 1, 1, pt_depth))
    up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
                   activation = 'linear', use_bias = False, weights = [up_c2_w])
    up_c2.trainable = False
    attn_layer = up_c2(attn_layer)
    mask_features = multiply([attn_layer, bn_features])
    gap_features = GlobalAveragePooling2D()(mask_features)
    gap_mask = GlobalAveragePooling2D()(attn_layer)
    # to account for missing values from the attention model
    gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
    gap_dr = Dropout(0.4)(gap)
    dr_steps = Dropout(0.4)(Dense(512, activation = 'swish', kernel_initializer="he_uniform")(gap_dr))
    out_layer = Dense(20, activation = 'softmax', kernel_initializer="he_uniform")(dr_steps)
    attn_model = Model(inputs = [pt_features], outputs = [out_layer], name = 'attention_model')

    attn_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                               metrics = ['acc'])
    
    print('Summary of Attention model only: ')
    print(attn_model.summary())
    
    

    
    tb_model = Sequential(name = 'combined_model')
    for layer in base_pretrained_model.layers[:len(base_pretrained_model.layers)-num_unfreeze]:
        layer.trainable = False
    for layer in base_pretrained_model.layers[len(base_pretrained_model.layers)-num_unfreeze:]:
        layer.trainable = True
    tb_model.add(base_pretrained_model)
    tb_model.add(attn_model)
    tb_model.compile(optimizer = Adam(lr = 0.001), loss = 'categorical_crossentropy',
                               metrics = [precision, recall, f1, 'acc'])
    print("Summary of final model: ")
    print(tb_model.summary())
    
    return tb_model

In [ ]:
model_resnet_0 = model_attention(shapes_resnet[0], num_unfreeze=14)
model_resnet_0.load_weights('../input/models-bird/resnet_original_v1_3.h5')
model_resnet_2 = model_attention(shapes_resnet[1], size=(224,224,3), num_unfreeze=20)
model_resnet_2.load_weights('../input/models-bird/resnet_cropped.h5')

In [ ]:
model_inception_0 = model_attention(shapes_inception[0], num_unfreeze=30)
model_inception_0.load_weights('../input/models-bird/inception_original_v1.h5')
model_inception_2 = model_attention(shapes_inception[1], size=(224,224,3), num_unfreeze=30)
model_inception_2.load_weights('../input/models-bird/inception_cropped.h5')

In [ ]:
img_width, img_height = 334, 334
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
for layer in base_model.layers[:len(base_model.layers)-17]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-17:]:
    layer.trainable = True
# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_0 = Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(0.0001)
model_0.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
             metrics=[precision, recall, f1, 'acc'])
model_0.load_weights('../input/models-bird/inception_v3_retrained_v2.h5')

In [ ]:
# dimensions of our images.
img_width, img_height = 224, 224
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
for layer in base_model.layers[:len(base_model.layers)-17]:
    layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-17:]:
    layer.trainable = True
# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_2 = Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(0.0001)
model_2.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
             metrics=[precision, recall, f1, 'acc'])
model_2.load_weights('../input/models-bird/inception_v3_cropped_retrained.h5')

# Stacking models

In [ ]:
members_original = [model_resnet_0, model_inception_0, model_0]
members_cropped = [model_resnet_2, model_inception_2, model_2]

In [ ]:
# update all layers in all models to not be trainable
for i in range(len(members_original)):
    model = members_original[i]
    for layer in model.layers:
        # make not trainable
        layer.trainable = False
    model = members_cropped[i]
    for layer in model.layers:
        # make not trainable
        layer.trainable = False

In [ ]:
from keras.layers.merge import concatenate
from keras.utils import plot_model
from sklearn.metrics import accuracy_score
from keras.layers import Input

### ORIGINAL

In [ ]:
inp1 = Input(shape=(20,), name="input1")
inp2 = Input(shape=(20,), name="input2")
inp3 = Input(shape=(20,), name="input3")
input_ = concatenate([inp1, inp2, inp3])
hidden = Dense(64, activation='relu')(input_)
output = Dense(20, activation='softmax')(hidden)
model = Model(inputs=[inp1, inp2, inp3], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images',
        target_size=(334, 334),
        batch_size=103,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')
input_X, input_Y = None, None
for X, y in validation_generator:
    input_X, input_Y = X, y
    break
predictions_resnet_0 = model_resnet_0.predict(input_X)
predictions_inception_0 = model_inception_0.predict(input_X)
predictions_0 = model_0.predict(input_X)



In [ ]:
res = [predictions_resnet_0, predictions_inception_0, predictions_0]
model.fit(res, input_Y, epochs=100, verbose=1)

### Cropped

In [ ]:
inp1 = Input(shape=(20,), name="input1")
inp2 = Input(shape=(20,), name="input2")
inp3 = Input(shape=(20,), name="input3")
input_ = concatenate([inp1, inp2, inp3])
hidden = Dense(64, activation='relu')(input_)
output = Dense(20, activation='softmax')(hidden)
model_crop = Model(inputs=[inp1, inp2, inp3], outputs=output)
model_crop.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
validation_datagen_crop = ImageDataGenerator(rescale=1./255)
validation_generator_crop = validation_datagen_crop.flow_from_directory(
        '../input/bird-dasat/bird_dataset/val_images_cropped',
        target_size=(224, 224),
        batch_size=92,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')
input_X, input_Y = None, None
for X, y in validation_generator_crop:
    input_X, input_Y = X, y
    break
predictions_resnet_2 = model_resnet_2.predict(input_X)
predictions_inception_2 = model_inception_2.predict(input_X)
predictions_2 = model_2.predict(input_X)

In [ ]:
res = [predictions_resnet_2, predictions_inception_2, predictions_2]
model_crop.fit(res, input_Y, epochs=150, verbose=1)

# Predictions

## Original

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        '../input/bird-dasat/bird_dataset/test_images',
        target_size=(334, 334),
        batch_size=517,
        class_mode=None,
        shuffle=False,
        interpolation='bicubic')
predictions_resnet_0 = model_resnet_0.predict(test_generator)
predictions_inception_0 = model_inception_0.predict(test_generator)
predictions_0 = model_0.predict(test_generator)
res = [predictions_resnet_0, predictions_inception_0, predictions_0]
predictions_original = model.predict(res)

## cropped

In [ ]:
test_datagen_crop = ImageDataGenerator(rescale=1./255)
test_generator_crop = test_datagen_crop.flow_from_directory(
        '../input/bird-dasat/bird_dataset/test_images_cropped',
        target_size=(224, 224),
        batch_size=459,
        class_mode=None,
        shuffle=False,
        interpolation='bicubic')
predictions_resnet_2 = model_resnet_2.predict(test_generator_crop)
predictions_inception_2 = model_inception_2.predict(test_generator_crop)
predictions_2 = model_2.predict(test_generator_crop)
res = [predictions_resnet_2, predictions_inception_2, predictions_2]
predictions_cropped = model_crop.predict(res)

## Predictions

In [ ]:
filenames_orig = [x.split('/')[1][:-4] for x in test_generator.filenames]
filenames_crop = [x.split('/')[1][:-4] for x in test_generator_crop.filenames]

In [ ]:
category = []
all_probas = []
for idx, elem in enumerate(filenames_orig):
    orig = (np.max(predictions_original[idx]), np.argmax(predictions_original[idx]))
    try:
        indice = filenames_crop.index(elem)
        crop = (np.max(predictions_cropped[indice]), np.argmax(predictions_cropped[indice]))
        all_proba = [orig, crop]
    except ValueError:
        all_proba = [orig]
    max_proba, cat = all_proba[0]
    for prob, label in all_proba[1:]:
        if prob > max_proba:
            max_proba = prob
            cat = label
    all_probas.append(all_proba)
    category.append(cat)

In [ ]:
df = pd.DataFrame({'Id': filenames_orig, 'Category': category})
df.head()

In [ ]:
df.to_csv('./submissions_final.csv', index=False)

SCORE : 0.84516